In [ ]:
import numpy as np
import pandas as pd

# Задание 1

Представьте, что мы решили попробовать реализовать рекомендательную систему на основе кластерного метода, ведь он довольно простой, поэтому в качестве MVP (Minimum Viable Product) может подойти.
> Требуется на основе имеющихся данных о просмотренных фильмах сделать рекомендации пользователю, для этого напишите функцию, моделирующую принцип работы кластерных рекомендаций, сделайте прогноз оценки пользователя `user_4` для фильма "Звёздные войны", используя евклидову и косинусную метрики. Попадёт ли этот фильм в рекомендацию если принять, что туда попадают фильмы с прогнозом оценки выше 3-х?

> Будем считать, что кластера всегда два, а фильм включается в рекомендацию если прогноз оценки выше 3-х. 

> На вход функция принимает данные оценок в виде датафрейма(`films_rating`), имя пользователя(`user_id`) в виде строки и название формулы для рассчёта меры схожести (`cos` или `euclid`). На выходе  датафрейм с прогнозом оценок. Если прогноз сделать невозможно, функция должна вывести сообщение об этом.

> Input:

*   `films_rating`: `pandas.dataframe`
*   `user_id`: `str`
*   `calculating_method`: `str`

> Output:

* `predict_rating`: `pandas.dataframe`

Создадим исходный датафрейм

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(4)
films_rating = pd.DataFrame(
    np.random.randint(0, 6, (6, 7)),
    index=['user_1', 'user_2', 'user_3', 'user_4', 'user_5', 'user_6'],
    columns=['Мстители', 'Матрица', 'Девчата', 
             'Начало', 'Пираты карибского моря', 'Звёздные войны', '1+1']).replace(0, np.nan)
films_rating

,Мстители,Матрица,Девчата,Начало,Пираты карибского моря,Звёздные войны,1+1
user_1,2,5.0,1.0,NaN,NaN,2.0,1
user_2,2,4.0,5.0,1.0,NaN,4.0,2
user_3,4,2.0,4.0,3.0,NaN,5.0,5
user_4,1,5.0,NaN,2.0,5.0,NaN,1
user_5,2,2.0,NaN,NaN,3.0,2.0,5
user_6,1,NaN,5.0,4.0,3.0,2.0,3


Напишем функцию по условию задачи

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances


def cluster_rec_sys(films_rating: pd.DataFrame, user_id: str, calculating_method: str) -> pd.core.frame.DataFrame:
  # Вычисляем векторное расстояние, которое будет характеризовать предпочтения пользователей,
  # в зависимости от переданного метода
  if calculating_method == 'euclid':
    distances = pd.DataFrame(euclidean_distances(films_rating.fillna(0).values),
                             index=films_rating.index.copy(), columns=films_rating.index.copy())
  elif calculating_method == 'cos':
    distances = pd.DataFrame(cosine_distances(films_rating.fillna(0).values),
                             index=films_rating.index.copy(), columns=films_rating.index.copy())
    
  # Делим эти расстояния на 2 кластера
  cluster_1 = distances.sort_values(user_id).iloc[:distances.shape[0] // 2, :]
  cluster_2 = distances.sort_values(user_id).iloc[distances.shape[0] // 2:, :]

  # Выделяем 2 кластера с фильмами
  films_cluster_1 = films_rating.T[list(cluster_1.index)].T
  films_cluster_2 = films_rating.T[list(cluster_2.index)].T

  # Проверяем, в каком из кластеров находится наш iser_id
  exists = user_id in films_cluster_1.index
  if exists == True:
    film_cluster_to_analys = films_cluster_1
  else:
    film_cluster_to_analys = films_cluster_2

  # Узнав, в каком из кластеров находится наш пользователь, просчитаем средние оценки других пользователей,
  # находящиеся в этом кластере(следовательно, имеющие схожие предпочтения к фильмам)
  mean_rating = film_cluster_to_analys.drop(user_id).mean(skipna=False)
  film_cluster_to_analys_T = film_cluster_to_analys.T
  film_cluster_to_analys_T['mean_rating'] = mean_rating

  # После того, как узнали средние оценки по каждому из фильмов, составим список рекомендаций из фильмов,
  # оценка которых выше 3.0
  recomended_films = []
  rating_list = []
  i = 0
  while i < len(film_cluster_to_analys_T["mean_rating"]):
    if film_cluster_to_analys_T["mean_rating"][i] > 3:
      recomended_films.append(film_cluster_to_analys_T.index[i])
      rating_list.append(film_cluster_to_analys_T["mean_rating"][i])
    i += 1
    
  # Проверяем, были ли вообще фильмы, оценки которых выше 3.0 и которые мы могли бы рекомендовать пользователю
  # Если такие были, создадим датафрейм с названиями этих фильмов и их оценками соотввественно
  if len(recomended_films) == 0:
    print('По данному датафрейму сделать прогноз невозможно')
    exit(0)
  else:
    predict_rating = pd.DataFrame({'film:': recomended_films, 'rating:': rating_list})
  return predict_rating


Пару тестов для задания 1:

In [ ]:
# cluster_rec_sys(films_rating=films_rating, user_id='user_4', calculating_method='euclid')
# cluster_rec_sys(films_rating=films_rating, user_id='user_1', calculating_method='euclid')
# cluster_rec_sys(films_rating=films_rating, user_id='user_4', calculating_method='cos')

,film:,rating:
0,Матрица,3.5


# Задание 2.

> Как уже упоминалось, рекомендательные системы также используются в сервисах типа онлайн магазин.

> К Вам обратилась компания, владеющая интернет-магазином, с задачей разработать рекомендательную систему на основе оценок, которые поставили пользователи.

> Требуется сделать рекомендацию конкретному пользователю, для этого напишите функцию, моделирующую принцип работы user based рекомендательной системы и сделать рекомендацию для пользователя `user_0`, используя косинусную метрику.

> На вход она должна принимать данные в виде массива с оценками пользователей (`products_ratings`), индекс пользователя (`user_id`), для которого необходимо сделать рекомендацию, и `calculating_method` - способ рассчёта схожести. На выходе названия 3-х рекомендуемых товаров, которые наиболее вероятно понравятся пользователю.

> Input:

*   `products_ratings`: `pandas.dataframe`
*   `user_id`: `str`
*   `calculating_method`: `str`

> Output:

* `predict_rating`: `pandas.dataframe`

Создадим исходный датафрейм

In [ ]:
np.random.seed(4)
data = np.random.randint(0, 5, (10, 15))
products_ratings = pd.DataFrame(data, 
                       columns=['Смартфон', 'Ноутбук', 'Наушники', 'Компьютерная мышь', 'Веб-камера', 
                                'Чехол для смартфона', 'Чехол для ноутбука', 'Портативная зарядка', 
                                'Флеш накопитель', 'Защитное стекло', 'Защитная плёнка', 'Зарядное устройство', 
                                'Монитор', 'Аудио система', 'Умная колонка'],
                       index=[f'user_{i}' for i in range(10)]).replace(0, np.nan)
products_ratings

,Смартфон,Ноутбук,Наушники,Компьютерная мышь,Веб-камера,Чехол для смартфона,Чехол для ноутбука,Портативная зарядка,Флеш накопитель,Защитное стекло,Защитная плёнка,Зарядное устройство,Монитор,Аудио система,Умная колонка
user_0,2.0,1.0,NaN,NaN,2.0,1.0,2,4.0,1.0,NaN,4.0,2.0,4.0,2.0,4.0
user_1,3.0,NaN,1.0,NaN,2.0,NaN,1,2.0,2.0,NaN,NaN,3.0,2.0,1.0,NaN
user_2,4.0,3.0,2.0,3.0,2.0,1.0,2,1.0,NaN,1.0,1.0,1.0,3.0,NaN,4.0
user_3,2.0,3.0,3.0,NaN,2.0,2.0,1,2.0,3.0,NaN,4.0,3.0,3.0,3.0,3.0
user_4,NaN,1.0,NaN,3.0,2.0,NaN,4,3.0,NaN,4.0,NaN,1.0,1.0,4.0,2.0
user_5,3.0,3.0,4.0,NaN,4.0,NaN,3,3.0,4.0,2.0,1.0,3.0,1.0,4.0,3.0
user_6,NaN,2.0,NaN,NaN,3.0,4.0,4,NaN,2.0,1.0,2.0,NaN,NaN,1.0,4.0
user_7,3.0,1.0,3.0,2.0,4.0,NaN,3,2.0,1.0,1.0,3.0,2.0,2.0,1.0,4.0
user_8,4.0,3.0,3.0,NaN,NaN,1.0,1,2.0,NaN,2.0,4.0,2.0,3.0,NaN,3.0
user_9,4.0,1.0,4.0,NaN,3.0,4.0,4,3.0,4.0,NaN,2.0,3.0,3.0,3.0,1.0


Напишем функцию по условию задачи

In [ ]:
from pandas.core.dtypes.missing import isna
from sklearn.metrics.pairwise import cosine_similarity


def user_based_rec_sys(products_ratings: pd.DataFrame, calculating_method: str, user_id: str) -> pd.core.frame.DataFrame:
  # Рассчитаем меру схожести пользователей исходя из имеющихся оценок, воспользовавшись косинусной мерой
  users_sim = cosine_similarity(products_ratings.fillna(0))
  users_sim_df = pd.DataFrame(users_sim, columns=products_ratings.index, index=products_ratings.index)

  # Рассчитаем суммарную меру для каждого пользователя, не забыв вычесть расстояние до самого себя,
  # то есть 1 в случае косинусного расстояния.
  users_sim_df['sum_cos_dist'] = np.sum(users_sim, axis=1) - 1

  # Умножить оценки пользователей на соответсвующие им косинусную меру,
  # относительно пользователя, для которого делается рекомендация.
  # Таким образом, оценки более «похожих» пользователей будут сильнее влиять на итоговую позицию товара
  # Нам нужны товары, которые еще не оценил выбранный пользователь, поэтому узнаем сначала их
  not_evaluated_goods = []
  for i in range(len(products_ratings.loc[user_id])):
    if isna(products_ratings.loc[user_id][i]) == True:
      not_evaluated_goods.append(products_ratings.loc[user_id].index[i])
  user_reformed_evaluations = products_ratings[not_evaluated_goods].fillna(0).T * users_sim_df.iloc[0, :-1]

  # Теперь для каждого товара рассчитаем сумму оценок, полученных на предыдущем шаге.
  user_reformed_evaluations['sum_score'] = user_reformed_evaluations.sum(axis=1)

  # Сумму, полученную на предыдущем шаге, разделим на сумму мер, полученную ранее (sum_cos_dist).
  # Таким образом, в датафрейме user_recomendations в строке result будут находиться прогнозы оценкок
  # на товары, которые пользователь еще не оценивал
  user_recomendations = user_reformed_evaluations.copy()
  user_recomendations['result'] = user_reformed_evaluations['sum_score'] / users_sim_df.sum_cos_dist[0]

  # Теперь отберем 3 товара, которые наиболее вероятно понравятся пользователю
  # Если таковых меньше или равно 3, выведем их все
  recomended_goods = []
  if len(user_recomendations['result']) <= 3:
    for i in range(len(user_recomendations['result'])):
      recomended_goods.append(user_recomendations['result'].index[i])
  else:
    for i in range(3):
      top_rating = max(user_recomendations['result'])
      recomended_good = user_recomendations[user_recomendations['result'] ==  top_rating].index[0]
      recomended_goods.append(recomended_good)
      user_recomendations.drop(labels = user_recomendations[user_recomendations['result'] ==  top_rating].index ,axis = 0, inplace = True)
  predict_rating = pd.DataFrame({'Рекомендуем купить вам следующие товары:': recomended_goods})
  return predict_rating

Пару тестов для задания 2:

In [ ]:
# # Ниже тесты для пользователей, для которых количество возможных товаров ниже или равно 3
# user_based_rec_sys(products_ratings=products_ratings, calculating_method='cos', user_id='user_7')
# user_based_rec_sys(products_ratings=products_ratings, calculating_method='cos', user_id='user_0')

# # Ниже тесты для пользователей, для которых количество возможных товаров привышает 3
# user_based_rec_sys(products_ratings=products_ratings, calculating_method='cos', user_id='user_1')

,Рекомендуем купить вам следующие товары:
0,Наушники
1,Компьютерная мышь
2,Защитное стекло
